**Задание**
1. Построить более сложную модель с подбором гиперпараметров
2. Проинтерпретировать полученную модель

Выполнено на основе "EDA предобработка данных"

In [ ]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 9.5 MB/s eta 0:00:00


In [ ]:
import optuna
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Загрузка данных и удаление строк с пропущенными значениями

In [ ]:
# Загрузка данных
df = pd.read_csv('/content/drive/MyDrive/animated_tv_series.csv', encoding='latin1')

# Удаление строк с пропущенными значениями
df.dropna(inplace=True)




Разделение данных на обучающий и тестовый наборы

In [ ]:
# Разделение данных на обучающий и тестовый наборы
X = df[['Episodes']]  # Признаки
y = df['IMDb']         # Целевая переменная
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



Определение гиперпараметров для оптимизации и обучение модели с выбранными гиперпараметрами

In [ ]:
def objective(trial):
    # Определение гиперпараметров для оптимизации
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }
    # Обучение модели с выбранными гиперпараметрами
    model = RandomForestRegressor(**params, random_state=42)
    model.fit(X_train, y_train)

    # Предсказание на тестовом наборе
    y_pred = model.predict(X_test)

    # Вычисление среднеквадратичной ошибки
    mse = mean_squared_error(y_test, y_pred)

    return mse


Создание и запуск оптимизации

In [ ]:
# Создание и запуск оптимизации
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)



[I 2024-05-08 10:08:45,051] A new study created in memory with name: no-name-f3779907-fcb5-4541-96ee-bf345ddea637
[I 2024-05-08 10:08:45,300] Trial 0 finished with value: 1.393482878203681 and parameters: {'n_estimators': 240, 'max_depth': 7, 'min_samples_split': 12, 'min_samples_leaf': 9, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 1.393482878203681.
[I 2024-05-08 10:08:45,681] Trial 1 finished with value: 1.4821139352233417 and parameters: {'n_estimators': 270, 'max_depth': 6, 'min_samples_split': 14, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': True}. Best is trial 0 with value: 1.393482878203681.
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
[I 

Вывод результатов оптимизации

In [ ]:
# Вывод результатов оптимизации
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value:  1.3617059396933542
  Params: 
    n_estimators: 104
    max_depth: 3
    min_samples_split: 8
    min_samples_leaf: 4
    max_features: log2
    bootstrap: True


Вывод показывает лучший результат оптимизации с помощью Optuna.

- `Value`: это значение среднеквадратичной ошибки (MSE) для лучшей модели.
- `Params`: это набор гиперпараметров, при которых было достигнуто лучшее значение ошибки.
Каждый параметр включает его название и значение.

`n_estimators` - указывает на количество деревьев в лесу

`max_depth` - максимальную глубину деревьев

`min_samples_split` - минимальное количество образцов, необходимое для разделения узла

`min_samples_leaf` - минимальное количество образцов, необходимое для разделения листа

`max_features` - количество признаков для разбиения на каждом узле

`bootstrap` - указывает на использование бутстрэпа в построении деревьев (True или False)

Обучаем модель из пердыдущей практики на основе полученных параметров

In [ ]:
# Разделение данных на обучающий и тестовый наборы
X = df[['Episodes']]  # Признаки
y = df['IMDb']         # Целевая переменная

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели случайного леса
rf_model = RandomForestRegressor(n_estimators=104, max_depth= 3, min_samples_split= 8, min_samples_leaf= 4, bootstrap= True)
rf_model.fit(X_train, y_train)

# Предсказание на тестовом наборе
y_pred = rf_model.predict(X_test)

# Оценка качества модели
rf_mse = mean_squared_error(y_test, y_pred)
print(f'Random Forest MSE: {rf_mse}')

Random Forest MSE: 1.3613294199449084
